<h1>Data Preparation - Polyphonic</h1>

In [1]:
import music21 as mu
import numpy as np
import pandas as pd
%pylab inline
import math
import json
from IPython.core.display import HTML
from IPython.core.display import Image
%load_ext music21.ipython21.ipExtension

Populating the interactive namespace from numpy and matplotlib


In [13]:
class CreateRawData:
    def __init__(self, title="", yearOfComposition="", yearOfPerformance="n/a", totalNumberOfMeasures=-1,
                composerName="", performerName="", baseAudioID="", globalCountStart=0, tempoForInTimeJazzSolos = -1,
                instrumentsOnScore = [], xmlFileString='none', numberOfQuarterNotesInBar=-1, genre="empty"):
        
        self.notesFromXML = mu.converter.parse(xmlFileString)
    
        self.tempoForInTimeJazzSolos = tempoForInTimeJazzSolos
        self.xmlFileString = xmlFileString
        self.title = title
        self.yearOfComposition = yearOfComposition
        self.yearOfPerformance = yearOfPerformance
        self.composerName = composerName
        self.performerName = performerName
        self.baseAudioID = baseAudioID
        self.instrumentsOnScore = instrumentsOnScore
        self.globalCountStart = globalCountStart
        self.numberOfQuarterNotesInBar = numberOfQuarterNotesInBar
        self.totalNumberOfMeasures = totalNumberOfMeasures
        self.genre = genre
        
        self.currentTempo = None
        self.textArray = None
        self.dynamicArray = None


        self.instrumentList = []
        
        
        self.midiList = []
        self.durationList = []
        self.locationList = []
        
        self.tempoList = []
        self.tempoListTwo = []
        self.scoreTextList = []
        self.scoreTextListTwo = []
        self.dynamicsList = []
        self.dynamicsListTwo = []
        self.indexList = []


        self.notesWithSlursAndTiesLeadingIn = []
        self.notesWithSlursAndTiesLeadingOut = []
        self.notes_list = []
        self.part_name_list = []    
        #self.instrumentArray = ['p1', 'p2'] 
        

        
        
        
        
    def getSummary(self):
        print 'BASIC INFO ABOUT SCORE:'
        print 'Composer: ', self.composerName
        print 'Title:', self.title
        print 'Year of Composition: ', self.yearOfComposition
        print 'Performer', self.performerName
        print 'Year of performance: ', self.yearOfPerformance
        print 'Instruments used: ', self.instrumentsOnScore
        
        print 'Alignment of Length: '
        print 'MIDI\t\t\t', len(self.midiList)
        print 'DURATION\t\t',  len(self.durationList)
        print 'TEMPO\t\t\t', len(self.tempoList)
        print 'DYNAMICS\t\t', len(self.dynamicsList)
        print 'SCORETEXT\t\t', len(self.scoreTextList)
        print 'CLEANED SCORETEXT\t',len(self.scoreTextList)
        print 'CLEANED DYNAMICS\t', len(self.dynamicsListTwo)
        print 'CLEANED TEMPO\t\t', len(self.tempoListTwo)
        print 'Beats in the bar: \t', self.lengthOfBar

        

    
    def extractIntoParts(self):
        #print self.notesFromXML
        a = 0
        self.parts_list = []
        
        while a < len(self.notesFromXML.parts):
            self.parts_list.append(self.notesFromXML.parts[a])
            a = a + 1
            
        for part in self.parts_list[0]:
            it = 0
            try:
                part.barDuration
                self.lengthOfBar = part.barDuration.quarterLength
                it = it + 1
                if it == 1:
                    break
            except:
                pass
        
        
        
        for part in self.parts_list:
            a = part.flat.getElementsByClass([mu.note.Note, mu.spanner.Slur, mu.spanner, mu.dynamics.Dynamic, mu.chord.Chord, mu.tempo.MetronomeMark, mu.expressions.TextExpression])
            #print part.flat.getElementsByClass(mu.note.Note)
            b = part.id
            self.notes_list.append(a)
            #print self.notes_list
            self.part_name_list.append(b)


                
    def processParts(self):
 
        self.i = 0
        for x,y in enumerate(self.instrumentsOnScore):
            for musicStuff in self.notes_list[x]:
                #print musicStuff
                #print musicStuff
#                 #IF ROW OF DATA IS SLUR OR TIE
#                 try:
#                     musicStuff.getFirst()
#                     print 'found'
#                     notesWithSlursAndTiesLeadingOut.append(musicStuff.getFirst())
#                     notesWithSlursAndTiesLeadingIn.append(musicStuff.getLast())
#                 except:
#                     pass


                try:
                    musicStuff.content
                    self.textArray = musicStuff.content
                    #print musicStuff.content
                except:
                    pass
                
                
                try:
                    musicStuff.volumeScalar
                    #print musicStuff.value
                    self.dynamicArray = musicStuff.value
                except:
                    pass

                # IF ROW OF DATA IS EXPRESSOIN



                # IF ROW OF DATA IS TEMPO
                try:
                    musicStuff.number
                    self.currentTempo = musicStuff.number

                except: 
                    pass

                #IF ROW OF DATA IS CHORD ARRAY
                try:    
                    musicStuff.intervalVector
                    for stuff in musicStuff.pitches:
                        #print 'Tempo', currentTempo, 'Chord', '\t\tmidi: ', stuff.midi, '\tOffset: ', musicStuff.offset , '\t\t\tDuration: ', musicStuff.duration.quarterLength, 'textArray', textArray
                        self.dynamicsList.append(self.dynamicArray)                     
                        self.midiList.append(stuff.midi)

                        
                        self.durationList.append(stuff.duration.quarterLength)
                        self.locationList.append(musicStuff.offset)
                        self.tempoList.append(self.currentTempo)
                        self.scoreTextList.append(self.textArray)
                        self.instrumentList.append(self.instrumentsOnScore[x])


                except:
                    pass

                #IF ROW OF DATA IS NOTE
                try: 
                    musicStuff.midi
                    #print 'Tempo', currentTempo , 'Note\tmidi:', musicStuff.midi, ' \tOffset:', musicStuff.offset, ' \t\tDuration:' , musicStuff.duration.quarterLength, 'textArray', textArray
                    self.dynamicsList.append(self.dynamicArray) 
                    self.midiList.append(musicStuff.midi)
                    self.durationList.append(musicStuff.duration.quarterLength)
                    self.locationList.append(musicStuff.offset)
                    self.tempoList.append(self.currentTempo)
                    self.scoreTextList.append(self.textArray)  
                    self.instrumentList.append(self.instrumentsOnScore[x])
  
                except:
                    pass
        rangeForIndex = len(self.dynamicsList)        
        self.indexList = range(rangeForIndex)
                
    def cleanUpForDynamicsAndScoreText(self):
        placeHolderForFirstDynamic = self.dynamicsList[0]
        placeHolderForFirstScoreText = self.scoreTextList[0]
        placeHolderForFirstTempo = self.tempoList[0]
        
        for i, things in enumerate(self.dynamicsList):
            if i > 0:
                if self.dynamicsList[i] != self.dynamicsList[i-1]:
                    self.dynamicsListTwo.append (self.dynamicsList[i])
                else:
                    self.dynamicsListTwo.append(None)
        self.dynamicsListTwo.insert(0,placeHolderForFirstDynamic)
        
        
        for i, things in enumerate(self.scoreTextList):
            if i > 0:
                if self.scoreTextList[i] != self.scoreTextList[i-1]:
                    self.scoreTextListTwo.append (self.scoreTextList[i])
                else:
                    self.scoreTextListTwo.append(None)
        self.scoreTextListTwo.insert(0,placeHolderForFirstScoreText)
        
        
        for i, things in enumerate(self.tempoList):
            if i > 0:
                if self.tempoList[i] != self.tempoList[i-1]:
                    self.tempoListTwo.append (self.tempoList[i])
                else:
                    self.tempoListTwo.append(None)
        self.tempoListTwo.insert(0,placeHolderForFirstTempo)
        
    def createDataFrameFromXML(self):
        rawMusicData = pd.DataFrame({'noteFrequency': self.midiList,
                   'noteDuration': self.durationList,
                   'noteLocation' : self.locationList,
                   #'currentTempo': self.tempoListTwo,
                   'currentTempo' : self.tempoForInTimeJazzSolos,
                   'scoreText': self.scoreTextListTwo,
                   'dynamics': self.dynamicsListTwo,
                   'instrument':self.instrumentList
                   })
        
        self.musicData = rawMusicData.sort(['noteLocation'])

        return self.musicData.reset_index(drop=True)
    
    
    def addOtherMetaDataToData(self, data):
        def calc_measure_floor(data_row):
            x = math.floor(data_row.noteLocation / self.numberOfQuarterNotesInBar) + 1
            return x
    
        data['currentMeasure'] = data.apply(calc_measure_floor, axis=1)
        def createUrlResourceString(data_row):
            url = 'http://api.musicdata.' + str(data_row['noteLocation']) + str(data_row['noteFrequency'])
            return url
    
        #data['resourceUrl'] = data.apply(createUrlResourceString, axis=1)
        
        def createAudioID(data_row):
            audioID = self.baseAudioID + str(data_row['noteLocation']) + str(data_row['noteFrequency'])
            return audioID
    
        data['audioID'] = data.apply(createAudioID, axis=1)
 
        data['ongoingDynamics'] = data['dynamics'].fillna(method="pad")
        data['ongoingTempo'] = data['currentTempo'].fillna(method="pad")
        data['ongoingScoretext'] = data['scoreText'].fillna(method="pad")
        
        def createNonChordScoreText(data_row):
            retValue = ''
            dynamics = data_row['ongoingDynamics']
            scoreText = data_row['ongoingScoretext']
            if dynamics is not None:
                retValue = dynamics
            if scoreText is not None:
                retValue = retValue + ', ' + scoreText
            return retValue
        
        if self.genre == 'nonJazz':
            data['scoreTextNonChord'] = data.apply(createNonChordScoreText, axis=1)
            data['scoreTextChord'] = None
        if self.genre == 'jazz':
            data['scoreTextChord'] = data['ongoingScoretext']
            data['scoreTextNonChord'] = None
        
        
        data['title'] = self.title
        data['composerName'] = self.composerName
        data['performerName'] = self.performerName
        data['yearOfComposition'] = self.yearOfComposition
        data['yearOfPerformance'] = self.yearOfPerformance
        
        dataForNoteCaculations = data.sort(['instrument', 'noteLocation'])
        
        #dataForNoteCaculations['previousNotePitchDistance'] = dataForNoteCaculations['noteFrequency'].shift(1)
        #dataForNoteCaculations['previousNotePitchDistance'] = dataForNoteCaculations['previousNotePitchDistance'] - dataForNoteCaculations['noteFrequency']
        
        #dataForNoteCaculations['nextNotePitchDistance'] = dataForNoteCaculations['noteFrequency'].shift(-1)
        #dataForNoteCaculations['nextNotePitchDistance'] = dataForNoteCaculations['nextNotePitchDistance'] - dataForNoteCaculations['noteFrequency']

        
        #dataForNoteCaculations['previousNoteDurationDistance'] = dataForNoteCaculations['noteLocation'].shift(1)
        #dataForNoteCaculations['previousNoteDurationDistance'] = abs(dataForNoteCaculations['previousNoteDurationDistance'] - dataForNoteCaculations['noteLocation'])
        
        #dataForNoteCaculations['nextNoteDurationDistance'] = dataForNoteCaculations['noteLocation'].shift(-1)
        #dataForNoteCaculations['nextNoteDurationDistance'] = abs(dataForNoteCaculations['nextNoteDurationDistance'] - dataForNoteCaculations['noteLocation'])
        
       
        
        dataForNoteCaculations['structuralNote'] = 1
        dataForNoteCaculations['ancillaryNote'] = 0
        dataForNoteCaculations['currentMeasureAsPercentageOfPieceLength'] = (dataForNoteCaculations['currentMeasure'] / self.totalNumberOfMeasures) * 100
     
        # new field for advanced mongodb queries
        dataForNoteCaculations['nextNoteFrequency'] = dataForNoteCaculations['noteFrequency'].shift(-1)
        dataForNoteCaculations['nextNoteLocation'] = dataForNoteCaculations['noteLocation'].shift(-1) 
        dataForNoteCaculations['nextNoteDuration'] = dataForNoteCaculations['noteDuration'].shift(-1)
        
        preparedData = dataForNoteCaculations.sort(['noteLocation'])
        
        
        preparedData['globalCount'] = self.indexList
        
        def manageGlobalCountAcrossCorpus(data_row):
            newGlobalCount = data_row['globalCount'] + self.globalCountStart
            return newGlobalCount
    
        preparedData['globalCount'] = preparedData.apply(manageGlobalCountAcrossCorpus, axis=1)   
        
        def getNextTenChords(data_row):
            currentIndex = data_row.name
            currentChord = data_row['scoreTextChord']
            listOfAllChords = preparedData['scoreText'].values
            sliceOfChords = listOfAllChords[currentIndex:]
            sliceOfChords = sliceOfChords[sliceOfChords != np.array(None)]
            sliceOfChords = sliceOfChords.tolist()
            sliceOfChords = sliceOfChords[0:10]
            sliceOfChords.insert(0,currentChord)
            sliceOfChords = '-'.join(sliceOfChords)
            return sliceOfChords
        
        preparedData['nextTenChords'] = preparedData.apply(getNextTenChords, axis=1)
        

        
        numberOfRowsInDataFrame = preparedData.shape[0]
        randomizingList = np.random.random_integers(0,1000000, (1000000,1))
        randomizingList = randomizingList.flatten().tolist()
        preparedData['randomizer'] = randomizingList[0:numberOfRowsInDataFrame]
        
        #preparedData = preparedData.sort('randomizer')
        
        return preparedData.reset_index(drop=True)
    
    
        
        

In [14]:
# fromXML = CreateRawData(title="Invention no. 13 in A minor", 
#                   globalCountStart = 0,
#                   yearOfComposition=1751, 
#                   yearOfPerformance=1751,
#                   composerName="Johann Sebastian Bach", 
#                   performerName="Johann Sebastian Bach", 
#                   baseAudioID="BACHINV13", 
#                   instrumentsOnScore = ['harpsichordTrebleClef', 'HarpsichordBassClef'],
#                   xmlFileString='FinalDataSet/BachInventionNo13.xml')

In [15]:
# fromXML = CreateRawData(title="String Quartet No. 3", 
#                   globalCountStart = 600,
#                   yearOfComposition=1841, 
#                   yearOfPerformance=1841,
#                   composerName="Ludwig Van Beethoven", 
#                   performerName="Ludwig Van Beethoven", 
#                   baseAudioID="BeethovenQ4OP9", 
#                   instrumentsOnScore = ['violin', 'violin2', 'viola', 'cello'],
#                   xmlFileString='corpus/String Quartet No 3 in G Major K156.mxl')

In [16]:
fromXML = CreateRawData(title="Stella By Starlight", 
                  numberOfQuarterNotesInBar = 4,
                  totalNumberOfMeasures = 161,
                  tempoForInTimeJazzSolos = 160,
                  globalCountStart = 20000,
                  yearOfComposition=1944, 
                  genre = "jazz",
                  yearOfPerformance= 1983,
                  composerName="Victor Young",
                  performerName="Keith Jarrett", 
                  baseAudioID="KJStellaByStarlight1983_1534_161", 
                  instrumentsOnScore = ['Piano Solo Jazz (Treble)', 'Jazz Harmony Bass Note'],
                  xmlFileString='FinalDataSet/XMLFiles/JazzSolos/KJAutumnLeaves1996_1371_192.xml')

In [17]:
fromXML.extractIntoParts()

In [18]:
fromXML.processParts()

In [19]:
fromXML.cleanUpForDynamicsAndScoreText()

In [20]:
fromXML.getSummary()

BASIC INFO ABOUT SCORE:
Composer:  Victor Young
Title: Stella By Starlight
Year of Composition:  1944
Performer Keith Jarrett
Year of performance:  1983
Instruments used:  ['Piano Solo Jazz (Treble)', 'Jazz Harmony Bass Note']
Alignment of Length: 
MIDI			1371
DURATION		1371
TEMPO			1371
DYNAMICS		1371
SCORETEXT		1371
CLEANED SCORETEXT	1371
CLEANED DYNAMICS	1371
CLEANED TEMPO		1371
Beats in the bar: 	4.0


In [21]:
fromXMLTwo = fromXML.createDataFrameFromXML()

In [22]:
fromXMLFinalDataset = fromXML.addOtherMetaDataToData(fromXMLTwo)

In [23]:
display(HTML(fromXMLFinalDataset[0:40].to_html()))

,currentTempo,dynamics,instrument,noteDuration,noteFrequency,noteLocation,scoreText,currentMeasure,audioID,ongoingDynamics,ongoingTempo,ongoingScoretext,scoreTextChord,scoreTextNonChord,title,composerName,performerName,yearOfComposition,yearOfPerformance,structuralNote,ancillaryNote,currentMeasureAsPercentageOfPieceLength,nextNoteFrequency,nextNoteLocation,nextNoteDuration,globalCount,nextTenChords,randomizer
0,160,None,Jazz Harmony Bass Note,4.0,60,0.0,C min7,1,KJStellaByStarlight1983_1534_1610.060,None,160,C min7,C min7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,0.621118,53,4.0,4.0,20000,C min7-C min7-F dom7-Bb maj7-Eb maj7-D dom7-G ...,25184
1,160,None,Piano Solo Jazz (Treble),0.5,67,1.0,None,1,KJStellaByStarlight1983_1534_1611.067,None,160,C min7,C min7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,0.621118,69,1.5,0.5,20001,C min7-F dom7-Bb maj7-Eb maj7-D dom7-G min7-G ...,771968
2,160,None,Piano Solo Jazz (Treble),0.5,69,1.5,None,1,KJStellaByStarlight1983_1534_1611.569,None,160,C min7,C min7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,0.621118,70,2.0,1.0,20002,C min7-F dom7-Bb maj7-Eb maj7-D dom7-G min7-G ...,68652
3,160,None,Piano Solo Jazz (Treble),1.0,70,2.0,None,1,KJStellaByStarlight1983_1534_1612.070,None,160,C min7,C min7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,0.621118,63,3.0,0.5,20003,C min7-F dom7-Bb maj7-Eb maj7-D dom7-G min7-G ...,100177
4,160,None,Piano Solo Jazz (Treble),0.5,63,3.0,None,1,KJStellaByStarlight1983_1534_1613.063,None,160,C min7,C min7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,0.621118,65,3.5,0.5,20004,C min7-F dom7-Bb maj7-Eb maj7-D dom7-G min7-G ...,249610
5,160,None,Piano Solo Jazz (Treble),0.5,65,3.5,None,1,KJStellaByStarlight1983_1534_1613.565,None,160,C min7,C min7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,0.621118,67,4.0,1.0,20005,C min7-F dom7-Bb maj7-Eb maj7-D dom7-G min7-G ...,51715
6,160,None,Jazz Harmony Bass Note,4.0,53,4.0,F dom7,2,KJStellaByStarlight1983_1534_1614.053,None,160,F dom7,F dom7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,1.242236,58,8.0,4.0,20006,F dom7-F dom7-Bb maj7-Eb maj7-D dom7-G min7-G ...,814401
7,160,None,Piano Solo Jazz (Treble),1.0,67,4.0,None,2,KJStellaByStarlight1983_1534_1614.067,None,160,F dom7,F dom7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,1.242236,69,5.0,0.5,20007,F dom7-Bb maj7-Eb maj7-D dom7-G min7-G dom7-C ...,600815
8,160,None,Piano Solo Jazz (Treble),0.5,69,5.0,None,2,KJStellaByStarlight1983_1534_1615.069,None,160,F dom7,F dom7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,1.242236,70,5.5,0.5,20008,F dom7-Bb maj7-Eb maj7-D dom7-G min7-G dom7-C ...,598296
9,160,None,Piano Solo Jazz (Treble),0.5,70,5.5,None,2,KJStellaByStarlight1983_1534_1615.570,None,160,F dom7,F dom7,None,Stella By Starlight,Victor Young,Keith Jarrett,1944,1983,1,0,1.242236,72,6.0,0.5,20009,F dom7-Bb maj7-Eb maj7-D dom7-G min7-G dom7-C ...,917054


In [77]:
fromXMLFinalDataset.max()

currentTempo                                  180.0
noteDuration                                    4.0
noteFrequency                                  89.0
noteLocation                                  906.5
currentMeasure                                227.0
ongoingTempo                                  180.0
yearOfComposition                            1950.0
yearOfPerformance                            2002.0
structuralNote                                  1.0
ancillaryNote                                   0.0
currentMeasureAsPercentageOfPieceLength       100.0
nextNoteFrequency                              89.0
nextNoteLocation                              906.5
nextNoteDuration                                4.0
globalCount                                 17160.0
randomizer                                 999315.0
dtype: float64

In [78]:
#fromXMLFinalDataset.to_json(orient='records', path_or_buf='beethovenStringQuartet3.json')

In [56]:
#fromXMLFinalDataset.to_json(orient='records', path_or_buf='bachInvention13.json')

In [79]:
#fromXMLFinalDataset.to_json(orient='records', path_or_buf='FinalDataSet/JSONFilesLoadedIntoDataBase/KJStellaByStarlight1983_1534_161.json')

In [45]:
fromXMLFinalDataset.columns

Index([u'currentTempo', u'dynamics', u'instrument', u'noteDuration', u'noteFrequency', u'noteLocation', u'scoreText', u'currentMeasure', u'audioID', u'ongoingDynamics', u'ongoingTempo', u'ongoingScoretext', u'scoreTextChord', u'scoreTextNonChord', u'title', u'composerName', u'performerName', u'yearOfComposition', u'yearOfPerformance', u'structuralNote', u'ancillaryNote', u'currentMeasureAsPercentageOfPieceLength', u'nextNoteFrequency', u'nextNoteLocation', u'nextNoteDuration', u'globalCount', u'nextTenChords', u'randomizer'], dtype='object')

In [13]:
#export note detail array to JSON

In [ ]:
# def createNoteDetail(data_row):
#     noteDetail = str(data_row['noteLocation'])  + ',' \
#     + str(data_row['noteFrequency']) + ',' \
#     + str(data_row['noteDuration'])
#     return noteDetail
    
# noteDetailArrayFromFrame['noteDetails'] = noteDetailArrayFromFrame.apply(createNoteDetail, axis=1)

# del noteDetailArrayFromFrame['noteDuration']
# del noteDetailArrayFromFrame['noteLocation']
# del noteDetailArrayFromFrame['noteFrequency']
noteDetailArrayFromFrame.to_json(orient='records', path_or_buf='polyphonicNoteSet.json')

In [ ]:
# XML Parsing functions
from xml.etree import ElementTree as ET

In [ ]:
xml = """
<?xml version='1.0' encoding='UTF-8'?>
<!DOCTYPE xgdresponse SYSTEM 'xgdresponse.dtd'>
<xgdresponse version='1.0'>
  <transid>2771709</transid>
  <errorcode>0</errorcode>
  <response>
    <result>
      <element>666</element>
      <errorcode>0</errorcode>
      <value>SOMETHING IMPORTANT!</value>
    </result>
  </response>
</xgdresponse>
""".strip()

value = ET.fromstring(xml).find('response/result/value')
if value is not None:
    print 'Found value:', value.text

In [ ]:
xmlNotes = ET.parse('corpus/mozartTesting5.xml')

In [ ]:
# val = xmlNotes.find('score-partwise')
# if val is not None:
#     print val.text
# else:
#     print ''

In [450]:
# for child in root:
#     for c2 in child:
#         for c3 in c2.findall('direction'):
#             for c4 in c3.findall('direction-type'):
#                 for c5 in c4.findall('rehearsal'):
#                     print c5.text
                
            